## 序列化与反序列化
- 二叉树 -> 字符串
- 字符串 -> 二叉树
- https://mp.weixin.qq.com/s/DVX2A1ha4xSecEXLxW_UsA
- https://leetcode-cn.com/problems/serialize-and-deserialize-binary-tree/solution/shou-hui-tu-jie-gei-chu-dfshe-bfsliang-chong-jie-f/

In [1]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [2]:
T = TreeNode(1)
T.left = TreeNode(2)
T.right = TreeNode(3)
T.right.left = TreeNode(4)
T.right.right = TreeNode(5)
T.left.left = None
T.left.right  = None
# T.left.right.left  = TreeNode(-1)
# T.left.right.right  = TreeNode(-1)

```
        1
    2       3
Na   Na   4   5
        Na Na Na Na 

```
- 特点：完美二叉树，每个有数的后面，都有节点接收

### 递归-序列化
- DFS（递归）
    - 前序遍历位置
    - 将二叉树构建成字符串

In [3]:
class Solution():
    # def __init__(self):
    #     # self.ls = []
    #     self.s = ""

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if root == None: 
            return 'X,'
        leftserilized = self.serialize(root.left)
        rightserilized = self.serialize(root.right)

        # '1,2,X,X,3,4,X,X,5,X,X,'
        subTree = str(root.val) + ',' + leftserilized + rightserilized
        return subTree

        # 这样写相当于前序遍历
        # '1,2,X,X,3,4,X,X,5,X,X,'
        return str(root.val) + ',' + leftserilized + rightserilized

In [4]:
s = Solution()
ls = s.serialize(T)
ls

'1,2,X,X,3,4,X,X,5,X,X,'

### 递归-反序列化
- DFS（递归）
    - 如果弹出的字符为 "X"，则返回 null 节点。
    - 如果弹出的字符是数值，则创建root节点，并递归构建root的左右子树，
    - 最后返回root。

In [16]:
s = '1,2,X,X,3,4,X,X,5,X,X,'
nodes = s.split(",")

```
        1
    2       3
Na   Na   4   5
        Na Na Na Na 

```

In [17]:
class Solution():
    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        # data = data.split(',')
        root = self.buildTree(data)
        return root 
    
    def buildTree(self, data):
        # 第一个元素是跟节点
        val = data.pop(0)
        if val == 'X': 
            return None
        node = TreeNode(val)

        # 为什么这里传入的是整个剩余的 字符串？？
        # 大局观，只要上面确定好根节点再做什么
        # 下面递归调用，就是对子树的重复构建
        node.left = self.buildTree(data)
        node.right = self.buildTree(data)
        return node

In [18]:
s = Solution()
ls = s.deserialize(nodes)
# ['1', '2', '4', '3']

### 如何使用递归的后序遍历方法序列化和反序列化二叉树

In [40]:
class Solution():
    def __init__(self):
        # self.ls = []
        self.s = ""

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if root == None: 
            self.s += 'X,'
            return 

        leftserilized = self.serialize(root.left)
        rightserilized = self.serialize(root.right)
        self.s += str(root.val)+","

        # 这样写相当于前序遍历
        return self.s
    
    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        data = data.split(',')
        data.pop()
        root = self.buildTree(data)
        return root 
    
    def buildTree(self, data):
        if len(data) == 0:
            return None

        # 第一个元素是根节点
        val = data.pop()
        if val == 'X': 
            return None
        node = TreeNode(val)

        #  为什么这里传入的是整个剩余的 字符串？？
        # 大局观，只要上面确定好根节点再做什么
        # 下面递归调用，就是对子树的重复构建
        node.right = self.buildTree(data)
        node.left = self.buildTree(data)
        
        return node

In [41]:
# s = Solution()
# s.serialize(T)

In [42]:
s = Solution()
ls = s.deserialize(s.serialize(T))

### 遍历-序列化
- BFS

- 维护一个队列，初始让根节点入列，考察出列节点：
    - 如果出列的节点是 null，将符号 'X' 推入 res 数组。
    - 如果出列的节点是数值，将节点值推入数组 res，并将它的左右子节点入列。
- 入列、出列…直到队列为空，就遍历完所有节点，res构建完毕，转成字符串就好。


In [26]:
class Solution():
    def __init__(self):
        self.ls = []
        self.q = []

    def serialize(self, root):
        if not root:
            # print(self.ls)
            return []

        self.q.append(root)
        res = ''
        while self.q:
            node = self.q.pop(0)
            if node != None:
                res += str(node.val) + ","
                self.q.append(node.left)
                self.q.append(node.right)
            else:
                res += "X,"
        return res


In [27]:
s = Solution()
s.serialize(T)

'1,2,3,X,X,4,5,X,X,X,X,'

In [28]:
nodes = '1,2,3,X,X,4,5,X,X,X,X,'

### 遍历-反序列化
- 依然先转成list数组，用一个指针 cursor 从第二项开始扫描。
- 起初，用list[0]构建根节点，并让根节点入列。
- 节点出列，此时 cursor 指向它的左子节点值，cursor+1 指向它的右子节点值。
    - 如果子节点值是数值，则创建节点，并认出列的父亲，同时自己也是父亲，入列。
    - 如果子节点值为 'X'，什么都不用做，因为出列的父亲的 left 和 right 本来就是 null
- 可见，所有的真实节点都会在队列里走一遍，出列就带出儿子入列


In [30]:
class Solution:
    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        if not data: 
            return None
        
        # 初始化队列，将 root 加入队列
        data = data.split(',')
        root = TreeNode(data.pop(0))
        q = [root]

        
        while q:
            node = q.pop(0)
            if data:
                val = data.pop(0)
                if val != 'X':
                    node.left = TreeNode(val)
                    q.append(node.left)
            if data:
                val = data.pop(0)
                if val != 'X':
                    node.right = TreeNode(val)
                    q.append(node.right)
        return root

In [32]:
s = Solution()
ls = s.deserialize(nodes)